In [ ]:
%pip install -q ipywidgets bqplot plotly colour-science

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
from glob import glob
#import class_interim

import colour
from colour.plotting import *
from colour.colorimetry import *
from colour.models import *
from colour import SDS_ILLUMINANTS

from matplotlib import rc
rc('text', usetex=True)

import plotly.express as px
from sklearn.metrics import pairwise_distances
from IPython.display import display, Latex

In [3]:
cd ../data/RS/

/home/gus/Documents/RCE/projects/2021-141_SdG_MBVB-Kusama/data/processed/RS


In [4]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": "Helvetica",
})

In [ ]:
files = sorted(glob('*txt*'))
ls = {'A':'-', 'B':'--', 'C':'-.', 'D':':'}
colors = {'A':'k', 'B':'blue', 'C':'yellow', 'D':'red'}

x = np.arange(0,10)
y = 2*x

fig, ax = plt.subplots(1,1,figsize=(16,8))
#sns.set()
ax.set_facecolor('0.9')
fs = 28

ax.plot(x,y)

plt.show()

# Groups 

In [ ]:
dic_spec = {'1':'D','2':'B','3':'C','4':'D','5':'B','6':'C','7a':'A','7b':'A','7c':'A','8a':'A','8b':'A'}
files = sorted(glob('*txt*'))

groups = [pd.read_csv(file, index_col='parameter').loc['group']['value_mean'][-2:] for file in files]
patroons = [pd.read_csv(file, index_col='parameter').loc['comments']['value_mean'].split('_')[0][7:] for file in files]
colours = [pd.read_csv(file, index_col='parameter').loc['color']['value_mean'] for file in files]

df_groups = pd.DataFrame(data={'groups':groups, 'patroons':patroons, 'spectra':[dic_spec[patroon] for patroon in patroons], 'colour': colours}).set_index('groups')

In [ ]:
df_groups.sort_values(by=['colour','groups'])

# Reproducibility - Group 1 

In [ ]:
file_mean = sorted(glob('*group01*'))[0]
df_mean = pd.read_csv(file_mean, index_col='parameter')
Lab_mean = df_mean.loc[['L*','a*','b*']]['value_mean'].astype(float).values

project = '2021-141'
Ids =['BEK1859P-r01','BEK1859P-r02','BEK1859P-r03','BEK1859P-r04']

Lab_values = [class_interim.itm(Id, project).get_data(data='Lab',show_plot=False).loc[['L*','a*','b*']]['value1'].astype(float).values for Id in Ids]
dE00s = []

for i in range(len(Lab_values)):
    Lab = Lab_values[i]
    dE00 = colour.delta_E(Lab_mean,Lab)
    dE00s.append(dE00)
    
MCDM = np.round(np.mean(dE00s),2)

print(f'The mean colour difference to the mean (MCDM) has a value of {MCDM} dE00.')
print(' ')
print('This means that colour differences smaller the 0.33 cannot be interpretated as changes within materials.')

# Repeatability - Group 2 

In [ ]:
file_mean = sorted(glob('*group02*'))[0]
df_mean = pd.read_csv(file_mean, index_col='parameter')
Lab_mean = df_mean.loc[['L*','a*','b*']]['value_mean'].astype(float).values

project = '2021-141'
Ids =['BEK1859P-r05','BEK1859P-r06','BEK1859P-r07','BEK1859P-r08']

Lab_values = [class_interim.itm(Id, project).get_data(data='Lab',show_plot=False).loc[['L*','a*','b*']]['value1'].astype(float).values for Id in Ids]
dE00s = []

for i in range(len(Lab_values)):
    Lab = Lab_values[i]
    dE00 = colour.delta_E(Lab_mean,Lab)
    dE00s.append(dE00)
    
MCDM = np.round(np.mean(dE00s),2)

print(f'The MCDM for the repeatability is {MCDM} dE00.')
print(' ')
print('This is a bit less interesting in our case. Reproducibility is more meaningful here.')

# Groups - Spectra

## reds

### Spectra

In [ ]:
files = sorted(glob('*txt*'))
ls = {'A':'-', 'B':'--', 'C':'-.', 'D':':'}
colors = {'A':'k', 'B':'blue', 'C':'yellow', 'D':'red'}


fig, ax = plt.subplots(1,1,figsize=(16,8))
#sns.set()
ax.set_facecolor('0.9')
fs = 28

df_RS = pd.DataFrame()

for file in files:
    df = pd.read_csv(file, index_col='parameter')
    color = df.loc['color']['value_mean']
    
    if color == 'red':
    
        group = df.loc['group']['value_mean'][-2:]
        patroon = df_groups.loc[group]['patroons']
        spectrum = df_groups.loc[group]['spectra']
        
        data_sp = df.loc['[MEASUREMENT DATA]':][6:].astype('float')

        wl = data_sp.index.astype('float')
        sp_mean = data_sp['value_mean']
        sp_std = data_sp['value_std']
        
        df_data = pd.DataFrame(data = {'groups':group,'spectra':spectrum, 'wl':wl.values, 'sp':sp_mean.values})
        df_RS = pd.concat([df_RS,df_data], axis=0)

        ax.plot(wl,sp_mean, lw = 3, ls = ls[spectrum], color='k', label=spectrum)
        ax.fill_between(wl,sp_mean+sp_std,sp_mean-sp_std, alpha=0.5, color='0.5', ec='none')
        
df_RS = df_RS.sort_values(by=['spectra','groups','wl'])

ax.set_xlim(400,740)
ax.xaxis.set_tick_params(labelsize=fs)
ax.yaxis.set_tick_params(labelsize=fs)

ax.set_xlabel('Wavelength (nm)', fontsize=fs)
ax.set_ylabel('Reflectance factor', fontsize=fs)

#handles, labels = ax.get_legend_handles_labels()
#labels = ['A','B','C','D']
#ax.legend(handles, labels, fontsize=fs, title='Spectra', title_fontsize=fs+2)
#ax.legend(fontsize=fs-3, title='Spectra', title_fontsize=fs+2)

plt.tight_layout()
plt.show()

In [ ]:
fig = px.line(df_RS, x ='wl',y='sp', line_group='groups', color='spectra') # 

fig.update_layout(
    autosize=False,
    width=1200,
    height=600,
    title="BEK 1858 P, red dots",
    xaxis_title="Wavelength (nm)",
    yaxis_title="Reflectance factor",
    legend_title="Spectra",
    font=dict(
        family="Droid Serif",
        size=22,
        color="Black"
    )
)

fig.show()

### Pairwise distance 

In [ ]:
df_grps = pd.DataFrame(df_RS.groups.unique())
spectra = (df_RS.sp.values.reshape(11, -1))
spderiv = np.diff(spectra, axis = 1)

reordered_grps = [0, 1, 2, 8, 9, 10, 3, 4, 5, 6, 7]
reordered_df = df_grps.iloc[reordered_grps,:]
reordered_sp = spectra[reordered_grps, :]
reordered_spderiv = spderiv[reordered_grps, :]

fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(20,10))
fs = 22

ax1.imshow(pairwise_distances(spectra, metric='cosine'))
ax2.imshow(pairwise_distances(reordered_sp, metric='cosine'))
ax3.imshow(pairwise_distances(reordered_spderiv, metric='cosine'))

ax1.set_xticks(np.arange(0,11))
ax1.set_xticklabels(df_grps.iloc[:,0].values, fontsize=fs)

ax1.set_yticks(np.arange(0,11))
ax1.set_yticklabels(df_grps.iloc[:,0].values, fontsize=fs)

ax1.set_xlabel('Groups', fontsize=fs)
ax1.set_ylabel('Groups', fontsize=fs)

ax1.set_title('Non ordered groups', fontsize=fs+2)



ax2.set_xticks(np.arange(0,11))
ax2.set_xticklabels(reordered_df.iloc[:,0].values, fontsize=fs)

ax2.set_yticks(np.arange(0,11))
ax2.set_yticklabels(reordered_df.iloc[:,0].values, fontsize=fs)

ax2.set_xlabel('Groups', fontsize=fs)
ax2.set_ylabel('Groups', fontsize=fs)

ax2.set_title('Reordered groups', fontsize=fs+2)


ax3.set_xticks(np.arange(0,11))
ax3.set_xticklabels(reordered_df.iloc[:,0].values, fontsize=fs)

ax3.set_yticks(np.arange(0,11))
ax3.set_yticklabels(reordered_df.iloc[:,0].values, fontsize=fs)

ax3.set_xlabel('Groups', fontsize=fs)
ax3.set_ylabel('Groups', fontsize=fs)

ax3.set_title('1st derivative - Reordered groups', fontsize=fs+2)


ax2.annotate('A', xy=(0.9,1.1), color='white', weight='bold', fontsize=fs)
ax2.annotate('B', xy=(6.85,7.15), color='white', weight='bold', fontsize=fs)
ax2.annotate('C', xy=(9.3,9.7), color='white', weight='bold', fontsize=fs)
ax2.annotate('D', xy=(3.8,4.1), color='white', weight='bold', fontsize=fs)

ax3.annotate('A', xy=(0.9,1.1), color='white', weight='bold', fontsize=fs)
ax3.annotate('B', xy=(6.85,7.15), color='white', weight='bold', fontsize=fs)
ax3.annotate('C', xy=(9.3,9.7), color='white', weight='bold', fontsize=fs)
ax3.annotate('D', xy=(3.8,4.1), color='white', weight='bold', fontsize=fs)

plt.tight_layout()
plt.show()

## whites 

In [ ]:
files = sorted(glob('*txt*'))
ls = {'B':'--', 'C':'-.'}


fig, ax = plt.subplots(1,1,figsize=(16,8))
fs = 28

df_final = pd.DataFrame()

for file in files:
    df = pd.read_csv(file, index_col='parameter')
    color = df.loc['color']['value_mean']
    
    if color == 'white':
    
        group = df.loc['group']['value_mean'][-2:]
        patroon = df_groups.loc[group]['patroons']
        spectrum = df_groups.loc[group]['spectra']
        
        data_sp = df.loc['[MEASUREMENT DATA]':][6:].astype('float')

        wl = data_sp.index.astype('float')
        sp_mean = data_sp['value_mean']
        sp_std = data_sp['value_std']
        
        df_data = pd.DataFrame(data = {'groups':group,'patroons':patroon,'spectra':spectrum, 'wl':wl.values, 'sp':sp_mean.values})
        df_final = pd.concat([df_final,df_data], axis=0)

        plot, = ax.plot(wl,sp_mean, color='k', lw = 3, ls = ls[spectrum], label=spectrum)
        ax.fill_between(wl,sp_mean+sp_std,sp_mean-sp_std, alpha=0.5, color='0.5', ec='none')


ax.set_xlim(400,740)
ax.xaxis.set_tick_params(labelsize=fs)
ax.yaxis.set_tick_params(labelsize=fs)


handles, labels = ax.get_legend_handles_labels()
labels = ['B','C']
ax.legend(handles, labels, fontsize=fs, title='Spectra', title_fontsize=fs+2)

plt.tight_layout()
plt.show()

# Groups - CIELAB 

In [ ]:
df_RS

In [ ]:
files = sorted(glob('*txt*'))
fmt = {'A':'o', 'B':'X', 'C':'s', 'D':'>'}


fig, ax = plt.subplots(2,2, figsize=(10, 10), gridspec_kw=dict(width_ratios=[1, 2], height_ratios=[2, 1]))

fs = 20
cps = 6     # capsize values
Lb = ax[0,0]
ab = ax[0,1]
AB = ax[1,0]
aL = ax[1,1]

df_Lab = pd.DataFrame()

for file in files:
    df = pd.read_csv(file, index_col='parameter')
    color = df.loc['color']['value_mean']
    
    if color == 'red':
    
        group = df.loc['group']['value_mean'][-2:]
        patroon = df_groups.loc[group]['patroons']
        spectrum = df_groups.loc[group]['spectra']        
               
        data_Lab = df.loc[['L*','a*','b*']].astype(float)
        
        L_mean = data_Lab.loc['L*']['value_mean']
        L_std = data_Lab.loc['L*']['value_std']
        
        a_mean = data_Lab.loc['a*']['value_mean']
        a_std = data_Lab.loc['a*']['value_std']
        
        b_mean = data_Lab.loc['b*']['value_mean']
        b_std = data_Lab.loc['b*']['value_std']
        
        df_data = pd.DataFrame(data = {'groups':group,'spectra':spectrum, 'wl':wl.values, 'L_mean':L_mean, 'a_mean':a_mean, 'b_mean':b_mean})
        df_Lab = pd.concat([df_Lab,df_data], axis=0)
        
        ab.errorbar(a_mean,b_mean, xerr=a_std, yerr=b_std,fmt=fmt[spectrum], capsize=5, ms=10, mec = 'none', mfc='k', ecolor='k')
        Lb.errorbar(L_mean,b_mean, xerr=L_std, yerr=b_std,fmt=fmt[spectrum], capsize=5, ms=10, mec = 'none', mfc='k', ecolor='k')
        aL.errorbar(a_mean,L_mean, xerr=a_std, yerr=L_std,fmt=fmt[spectrum], capsize=5, ms=10, mec = 'none', mfc='k', ecolor='k')
        


#ax.set_xlim(400,740)
ab.xaxis.set_tick_params(labelsize=fs)
ab.yaxis.set_tick_params(labelsize=fs)
Lb.xaxis.set_tick_params(labelsize=fs)
Lb.yaxis.set_tick_params(labelsize=fs)
aL.xaxis.set_tick_params(labelsize=fs)
aL.yaxis.set_tick_params(labelsize=fs)


Lb.set_xlabel("CIE $L^*$", fontsize=fs)
Lb.set_ylabel("CIE $b^*$", fontsize=fs)
ab.set_xlabel("CIE $a^*$", fontsize=fs)
ab.set_ylabel("CIE $b^*$", fontsize=fs)
aL.set_xlabel("CIE $a^*$", fontsize=fs)
aL.set_ylabel("CIE $L^*$", fontsize=fs) 


plt.tight_layout()
plt.show()

In [ ]:
df_Lab

In [ ]:
import plotly.graph_objects as go

# Create data for the graph
x_values = [1, 2, 3, 4, 5]
y_values_1 = [10, 20, 30, 40, 50]
y_values_2 = [5, 10, 15, 20, 25]

# Create a plot for each line
line_1 = go.Scatter(x=x_values, y=y_values_1, name='Line 1')
line_2 = go.Scatter(x=x_values, y=y_values_2, name='Line 2')

# Combine the plot into a single figure
fig = go.Figure(data=[line_1, line_2])

# Add a button to toggle each line on/off
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            buttons=list([
                dict(label="Line 1",
                     method="update",
                     args=[{"visible": [True, False]},
                           {"title": "Line 1"}]),
                dict(label="Line 2",
                     method="update",
                     args=[{"visible": [False, True]},
                           {"title": "Line 2"}]),
                dict(label="Both",
                     method="update",
                     args=[{"visible": [True, True]},
                           {"title": "Both Lines"}])
            ]),
        )
    ]
)

# Show the graph
fig.show()


In [ ]:
import plotly.graph_objects as go

# Create data for the graph
x_values = [1, 2, 3, 4, 5]
y_values_1 = [10, 20, 30, 40, 50]
y_values_2 = [5, 10, 15, 20, 25]
y_values_3 = [15, 25, 35, 45, 55]
y_values_4 = [20, 30, 40, 50, 60]
y_values_5 = [25, 35, 45, 55, 65]
y_values_6 = [30, 40, 50, 60, 70]

# Create a plot for each line
line_1 = go.Scatter(x=x_values, y=y_values_1, name='Line 1')
line_2 = go.Scatter(x=x_values, y=y_values_2, name='Line 2')
line_3 = go.Scatter(x=x_values, y=y_values_3, name='Line 3')
line_4 = go.Scatter(x=x_values, y=y_values_4, name='Line 4')
line_5 = go.Scatter(x=x_values, y=y_values_5, name='Line 5')
line_6 = go.Scatter(x=x_values, y=y_values_6, name='Line 6')

# Group the lines into three groups
group_1 = [line_1, line_2, line_3]
group_2 = [line_4, line_5, line_6]

# Combine the plot into a single figure
fig = go.Figure(data=group_1 + group_2)

# Add buttons to toggle each group on/off
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            buttons=list([
                dict(label="Group 1",
                     method="update",
                     args=[{"visible": [True, True, True, False, False, False]},
                           {"title": "Group 1"}]),
                dict(label="Group 2",
                     method="update",
                     args=[{"visible": [False, False, False, True, True, True]},
                           {"title": "Group 2"}]),
                dict(label="Both",
                     method="update",
                     args=[{"visible": [True, True, True, True, True, True]},
                           {"title": "Both Groups"}])
            ]),
        )
    ]
)

# Show the graph
fig.show()


In [ ]:
import plotly.graph_objects as go

# Create data for the graph
x_values = [1, 2, 3, 4, 5]
y_values_1 = [10, 20, 30, 40, 50]
y_values_2 = [5, 10, 15, 20, 25]
y_values_3 = [15, 25, 35, 45, 55]
y_values_4 = [20, 30, 40, 50, 60]
y_values_5 = [25, 35, 45, 55, 65]
y_values_6 = [30, 40, 50, 60, 70]

# Define a color for each group
group_1_color = 'blue'
group_2_color = 'red'

# Create a plot for each line
line_1 = go.Scatter(x=x_values, y=y_values_1, name='Line 1', line=dict(color=group_1_color, dash = 'dash'))
line_2 = go.Scatter(x=x_values, y=y_values_2, name='Line 2', line=dict(color=group_1_color))
line_3 = go.Scatter(x=x_values, y=y_values_3, name='Line 3', line=dict(color=group_1_color))
line_4 = go.Scatter(x=x_values, y=y_values_4, name='Line 4', line=dict(color=group_2_color))
line_5 = go.Scatter(x=x_values, y=y_values_5, name='Line 5', line=dict(color=group_2_color))
line_6 = go.Scatter(x=x_values, y=y_values_6, name='Line 6', line=dict(color=group_2_color))

# Group the lines into two groups
group_1 = [line_1, line_2, line_3]
group_2 = [line_4, line_5, line_6]

# Combine the plot into a single figure
fig = go.Figure(data=group_1 + group_2)

# Add buttons to toggle each group on/off
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            buttons=list([
                dict(label="Group 1",
                     method="update",
                     args=[{"visible": [True, True, True, False, False, False]},
                           {"title": "Group 1"}]),
                dict(label="Group 2",
                     method="update",
                     args=[{"visible": [False, False, False, True, True, True]},
                           {"title": "Group 2"}]),
                dict(label="Both",
                     method="update",
                     args=[{"visible": [True, True, True, True, True, True]},
                           {"title": "Both Groups"}])
            ]),
        )
    ]
)

# Show the graph
fig.show()


In [ ]:
from ipywidgets import Layout, Button, Box, interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.widgets as widgets

# Create data for the graph
x_values = [1, 2, 3, 4, 5]
y_values_1 = [10, 20, 30, 40, 50]
y_values_2 = [5, 10, 15, 20, 25]
y_values_3 = [15, 25, 35, 45, 55]
y_values_4 = [20, 30, 40, 50, 60]
y_values_5 = [25, 35, 45, 55, 65]
y_values_6 = [30, 40, 50, 60, 70]

# Define a color for each group
group_1_color = 'blue'
group_2_color = 'red'

# Create a figure and axis object
fig, ax = plt.subplots()

# Create each line and set its color
line_1, = ax.plot(x_values, y_values_1, label='Line 1', color=group_1_color)
line_2, = ax.plot(x_values, y_values_2, label='Line 2', color=group_1_color)
line_3, = ax.plot(x_values, y_values_3, label='Line 3', color=group_1_color)
line_4, = ax.plot(x_values, y_values_4, label='Line 4', color=group_2_color)
line_5, = ax.plot(x_values, y_values_5, label='Line 5', color=group_2_color)
line_6, = ax.plot(x_values, y_values_6, label='Line 6', color=group_2_color)

# Define a function to toggle the visibility of a group of lines
def toggle_group(group, visible):
    for line in group:
        line.set_visible(visible)

# Create a dictionary of group names and the lines in each group
groups = {
    'Group 1': [line_1, line_2, line_3],
    'Group 2': [line_4, line_5, line_6]
}

# Create a dictionary of check boxes, with the group name as the key and the 
# callback function as the value
checkboxes = {}
for group_name, group_lines in groups.items():
    checkboxes[group_name] = widgets.CheckButtons(ax=ax, labels=group_name)

    def callback(label, group=group_lines):
        visible = not group[0].get_visible()
        toggle_group(group, visible)
        plt.draw()

    checkboxes[group_name].on_clicked(callback)

# Add a button to toggle all lines on or off
checkboxes['All'] = widgets.CheckButtons(ax=ax, labels=['All'])

def callback(label):
    visible = not line_1.get_visible()
    for line in [line_1, line_2, line_3, line_4, line_5, line_6]:
        line.set_visible(visible)
    plt.draw()

checkboxes['All'].on_clicked(callback)

# Add a subplot tool to the plot
subplot_tool = widgets.SubplotTool(targetfig=fig, geometry="111")
plt.subplots_adjust(bottom=0.2)
plt.show()
